In [196]:
import pandas as pd
chemin=r"/Users/franckglastre/"
df=pd.read_csv(chemin+"emaildata_100000_0.csv")
df.columns

Index(['Unnamed: 0', 'date', 'sender', 'recipient1', 'subject', 'text'], dtype='object')

In [197]:
df=df.iloc[0:100,]

In [174]:
type("".join(df.text[11]))

str

In [175]:
# exploration un email avant traitement dans colonne text, ici ligne 12
df.text[12]

'[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/09/2000 \', \'02:16 PM ---------------------------\', \'\', \'\', \'Richard Burchfield\', \'10/06/2000 06:59 AM\', \'To: Phillip K Allen/HOU/ECT@ECT\', \'cc: Beth Perlman/HOU/ECT@ECT \', \'Subject: Consolidated positions: Issues & To Do list\', \'\', \'Phillip,\', \' Below is the issues & to do list as we go forward with documenting the \', \'requirements for consolidated physical/financial positions and transport \', "trade capture. What we need to focus on is the first bullet in Allan\'s list; ", \'the need for a single set of requirements. Although the meeting with Keith, \', \'on Wednesday,  was informative the solution of creating a infinitely dynamic \', \'consolidated position screen, will be extremely difficult and time \', \'consuming.  Throughout the meeting on Wednesday, Keith alluded to the \', \'inability to get consensus amongst the traders on the presentation of the \', \'consolidated position, s

In [176]:
# netoyage partiel avec tokenisation phrase pour test

import re
r = re.compile(r"[\\\''\",]")
txt = df.text[12]
texte=r.sub('', txt)

from nltk.tokenize import sent_tokenize
artikle=sent_tokenize(texte)

print(artikle)

['[ ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/09/2000  02:16 PM ---------------------------   Richard Burchfield 10/06/2000 06:59 AM To: Phillip K Allen/HOU/ECT@ECT cc: Beth Perlman/HOU/ECT@ECT  Subject: Consolidated positions: Issues & To Do list  Phillip  Below is the issues & to do list as we go forward with documenting the  requirements for consolidated physical/financial positions and transport  trade capture.', 'What we need to focus on is the first bullet in Allans list;  the need for a single set of requirements.', 'Although the meeting with Keith  on Wednesday  was informative the solution of creating a infinitely dynamic  consolidated position screen will be extremely difficult and time  consuming.', 'Throughout the meeting on Wednesday Keith alluded to the  inability to get consensus amongst the traders on the presentation of the  consolidated position so the solution was to make it so that a trader can  arrange the position screen to their liking (mu

In [201]:
#application au dataframe

'''
def nettoyage(df,champ):
    import re
    from nltk.tokenize import sent_tokenize
    r = re.compile(r"[\\\''\",]")
    df=df.apply(lambda x:sent_tokenize(r.sub('', x[champ]))
    return (df)

df1=nettoyage(df,"text")

  
import re
from nltk.tokenize import sent_tokenize
r = re.compile(r"[\\\''\",]")
#df=df.apply(lambda x:r.sub('',x['text']),axis=1) 
df["text2"]=df["text"].apply(lambda x:sent_tokenize(r.sub('',x)))
''' 
import re
from nltk.tokenize import sent_tokenize,word_tokenize


def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico

df['text2']=df['text'].apply(lambda x:dictionarize(x))

In [205]:
df.iloc[11:13,]['text2'][12]

{0: ['forwarded',
  'by',
  'phillip',
  'k',
  'allen',
  'hou',
  'ect',
  'on',
  'pm',
  'richard',
  'burchfield',
  'am',
  'to',
  'phillip',
  'k',
  'allen',
  'hou',
  'ect',
  'ect',
  'cc',
  'beth',
  'perlman',
  'hou',
  'ect',
  'ect',
  'subject',
  'consolidated',
  'positions',
  'issues',
  'to',
  'do',
  'list',
  'phillip',
  'below',
  'is',
  'the',
  'issues',
  'to',
  'do',
  'list',
  'as',
  'we',
  'go',
  'forward',
  'with',
  'documenting',
  'the',
  'requirements',
  'for',
  'consolidated',
  'physical',
  'financial',
  'positions',
  'and',
  'transport',
  'trade',
  'capture'],
 1: ['what',
  'we',
  'need',
  'to',
  'focus',
  'on',
  'is',
  'the',
  'first',
  'bullet',
  'in',
  'allan',
  's',
  'list',
  'the',
  'need',
  'for',
  'a',
  'single',
  'set',
  'of',
  'requirements'],
 2: ['although',
  'the',
  'meeting',
  'with',
  'keith',
  'on',
  'wednesday',
  'was',
  'informative',
  'the',
  'solution',
  'of',
  'creating',
  '

In [207]:
import pandas as pd
import numpy as np



def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['text2']
    artikle = corpus[i]
    texte=df['text'][i]
    
    
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
    
    return summary

In [214]:
# texte fonction de résumé sur la ligne 12
summarize(12, 5, df)

["', 'Valuation of Gas Daily options.",
 "', 'Inclusion of positions for exotic options currently managed in spreadsheets.",
 "', 'Ability to isolate the position change due to changed deals in the position ', 'manager.",
 "', 'Eliminate change deals with no economic impact from the TDS deal ticker.",
 "', 'Deployment of TDS in Canada."]

In [221]:
df['summary']=df.apply(lambda x:summarize(x.keys(),5,df),axis=1)

KeyError: "None of [Index(['Unnamed: 0', 'date', 'sender', 'recipient1', 'subject', 'text',\n       'text2', 'summary'],\n      dtype='object')] are in the [index]"

In [220]:
df['summary']=df.apply(lambda x:x.keys(),axis=1)